<a href="https://colab.research.google.com/github/kuds/rl-connect-four/blob/main/%5BConnect%20Four%5D%20Self%20Play.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Playing Connect Four using Self Play

In [1]:
!pip install ray[rllib] gputil open_spiel

In [2]:
import functools
import numpy as np
from ray.air.constants import TRAINING_ITERATION
from ray.rllib.core.rl_module.default_model_config import DefaultModelConfig
from ray.rllib.core.rl_module.multi_rl_module import MultiRLModuleSpec
from ray.rllib.core.rl_module.rl_module import RLModuleSpec
from ray.rllib.env.utils import try_import_pyspiel, try_import_open_spiel
from ray.rllib.env.wrappers.open_spiel import OpenSpielEnv
from ray.rllib.examples.rl_modules.classes.random_rlm import RandomRLModule
from ray.rllib.examples.multi_agent.utils import (
    ask_user_for_action,
    SelfPlayCallback,
    SelfPlayCallbackOldAPIStack,
)
from ray.rllib.examples._old_api_stack.policy.random_policy import RandomPolicy
from ray.rllib.policy.policy import PolicySpec
from ray.rllib.utils.metrics import NUM_ENV_STEPS_SAMPLED_LIFETIME
from ray.rllib.utils.test_utils import (
    add_rllib_example_script_args,
    run_rllib_example_script_experiment,
)
from ray.tune.registry import get_trainable_cls, register_env
import platform

import torch
from importlib.metadata import version

# Import after try_import_open_spiel, so we can error out with hints.
from open_spiel.python.rl_environment import Environment  # noqa: E402

In [3]:
print(f"Python Version: {platform.python_version()}")
print(f"Torch Version: {version('torch')}")
print(f"Is Cuda Available: {torch.cuda.is_available()}")
print(f"Cuda Version: {torch.version.cuda}")
print(f"Numpy Version: {version('numpy')}")
print(f"Stable Baselines3 Version: {version('open_spiel')}")
print(f"Stable Baselines3 Version: {version('ray')}")

Python Version: 3.10.12
Torch Version: 2.5.0+cu121
Is Cuda Available: False
Cuda Version: 12.1
Numpy Version: 1.26.4
Stable Baselines3 Version: 1.5
Stable Baselines3 Version: 2.38.0


In [4]:
open_spiel = try_import_open_spiel(error=True)
pyspiel = try_import_pyspiel(error=True)

In [5]:
class Config:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
        self.env = "connect_four"
        self.framework = "torch"
        self.algo = "PPO"
        self.num_gpus = 0
        self.num_env_runners = 2
        self.enable_new_api_stack = True
        self.min_league_size = 3
        self.win_rate_threshold = 0.95
        self.num_episodes_human_play = 10
        self.from_checkpoint = None
        self.stop_iters = 100
        self.stop_timesteps = 1000000
        self.as_release_test = False
        self.num_cpus = 1
        self.local_mode = False
        self.evaluation_interval = 10
        self.evaluation_num_episodes = 10
        self.evaluation_num_env_runners = 1
        self.evaluation_duration = 10
        self.evaluation_duration_unit = 10
        self.evaluation_parallel_to_training = False
        self.evaluation_config = {}
        self.evaluation_num_workers = 1
        self.log_level = "INFO"
        self.output = "ray_results"
        self.no_tune = False

config_args = Config()

In [6]:
# parser = add_rllib_example_script_args(default_timesteps=2000000)
# parser.set_defaults(env="connect_four")
# parser.add_argument(
#     "--win-rate-threshold",
#     type=float,
#     default=0.95,
#     help="Win-rate at which we setup another opponent by freezing the "
#     "current main policy and playing against a uniform distribution "
#     "of previously frozen 'main's from here on.",
# )
# parser.add_argument(
#     "--min-league-size",
#     type=float,
#     default=3,
#     help="Minimum number of policies/RLModules to consider the test passed. "
#     "The initial league size is 2: `main` and `random`. "
#     "`--min-league-size=3` thus means that one new policy/RLModule has been "
#     "added so far (b/c the `main` one has reached the `--win-rate-threshold "
#     "against the `random` Policy/RLModule).",
# )
# parser.add_argument(
#     "--num-episodes-human-play",
#     type=int,
#     default=10,
#     help="How many episodes to play against the user on the command "
#     "line after training has finished.",
# )
# parser.add_argument(
#     "--from-checkpoint",
#     type=str,
#     default=None,
#     help="Full path to a checkpoint file for restoring a previously saved "
#     "Algorithm state.",
# )

# args = parser.parse_args()

In [7]:
register_env("open_spiel_env", lambda _: OpenSpielEnv(pyspiel.load_game(config_args.env)))

def agent_to_module_mapping_fn(agent_id, episode, **kwargs):
  # agent_id = [0|1] -> module depends on episode ID
  # This way, we make sure that both modules sometimes play agent0
  # (start player) and sometimes agent1 (player to move 2nd).
  return "main" if hash(episode.id_) % 2 == agent_id else "random"

def policy_mapping_fn(agent_id, episode, worker, **kwargs):
  return "main" if episode.episode_id % 2 == agent_id else "random"

config = (
get_trainable_cls(config_args.algo)
.get_default_config()
.api_stack(
    enable_rl_module_and_learner=config_args.enable_new_api_stack,
    enable_env_runner_and_connector_v2=config_args.enable_new_api_stack,
)
.environment("open_spiel_env")
.framework(config_args.framework)
# Set up the main piece in this experiment: The league-bases self-play
# callback, which controls adding new policies/Modules to the league and
# properly matching the different policies in the league with each other.
.callbacks(
    functools.partial(
        (
            SelfPlayCallback
            if config_args.enable_new_api_stack
            else SelfPlayCallbackOldAPIStack
        ),
        win_rate_threshold=config_args.win_rate_threshold,
    )
)
.env_runners(
    num_env_runners=(config_args.num_env_runners or 2),
    num_envs_per_env_runner=1 if config_args.enable_new_api_stack else 5,
)
.learners(
    num_learners=config_args.num_gpus,
    num_gpus_per_learner=1 if config_args.num_gpus else 0,
)
.resources(
    num_cpus_for_main_process=1,
)
.multi_agent(
    # Initial policy map: Random and default algo one. This will be expanded
    # to more policy snapshots taken from "main" against which "main"
    # will then play (instead of "random"). This is done in the
    # custom callback defined above (`SelfPlayCallback`).
    policies=(
        {
            # Our main policy, we'd like to optimize.
            "main": PolicySpec(),
            # An initial random opponent to play against.
            "random": PolicySpec(policy_class=RandomPolicy),
        }
        if not config_args.enable_new_api_stack
        else {"main", "random"}
    ),
    # Assign agent 0 and 1 randomly to the "main" policy or
    # to the opponent ("random" at first). Make sure (via episode_id)
    # that "main" always plays against "random" (and not against
    # another "main").
    policy_mapping_fn=(
        agent_to_module_mapping_fn
        if config_args.enable_new_api_stack
        else policy_mapping_fn
    ),
    # Always just train the "main" policy.
    policies_to_train=["main"],
)
.rl_module(
    model_config=DefaultModelConfig(fcnet_hiddens=[512, 512]),
    rl_module_spec=MultiRLModuleSpec(
        rl_module_specs={
            "main": RLModuleSpec(),
            "random": RLModuleSpec(module_class=RandomRLModule),
        }
    ),
)
)

# Only for PPO, change the `num_epochs` setting.
if config_args.algo == "PPO":
  config.training(num_epochs=20)

stop = {
  NUM_ENV_STEPS_SAMPLED_LIFETIME: config_args.stop_timesteps,
  TRAINING_ITERATION: config_args.stop_iters,
  "league_size": config_args.min_league_size,
}

# Train the "main" policy to play really well using self-play.
results = None
if not config_args.from_checkpoint:
    results = run_rllib_example_script_experiment(config, config_args, stop=stop)

# Restore trained Algorithm (set to non-explore behavior) and play against
# human on command line.
if config_args.num_episodes_human_play > 0:
    num_episodes = 0
    config.explore = False
    algo = config.build()
    if config_args.from_checkpoint:
        algo.restore(config_args.from_checkpoint)
    else:
        checkpoint = results.get_best_result().checkpoint
        if not checkpoint:
            raise ValueError("No last checkpoint found in results!")
        algo.restore(checkpoint)

    # Play from the command line against the trained agent
    # in an actual (non-RLlib-wrapped) open-spiel env.
    human_player = 1
    env = Environment(config_args.env)

    while num_episodes < config_args.num_episodes_human_play:
        print("You play as {}".format("o" if human_player else "x"))
        time_step = env.reset()
        while not time_step.last():
            player_id = time_step.observations["current_player"]
            if player_id == human_player:
                action = ask_user_for_action(time_step)
            else:
                obs = np.array(time_step.observations["info_state"][player_id])
                action = algo.compute_single_action(obs, policy_id="main")
                # In case computer chooses an invalid action, pick a
                # random one.
                legal = time_step.observations["legal_actions"][player_id]
                if action not in legal:
                    action = np.random.choice(legal)
            time_step = env.step([action])
            print(f"\n{env.get_state}")

        print(f"\n{env.get_state}")

        print("End of game!")
        if time_step.rewards[human_player] > 0:
            print("You win")
        elif time_step.rewards[human_player] < 0:
            print("You lose")
        else:
            print("Draw")
        # Switch order of players.
        human_player = 1 - human_player

        num_episodes += 1

    algo.stop()

/usr/local/lib/python3.10/dist-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:164: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:188: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
2024-10-25 02:59:54,737	INFO worker.py:1816 -- Started a local Ray instance.


AttributeError: 'Config' object has no attribute 'no_tune'